In [11]:
""""
E91 protocol with eve existence
iQuHACK 2022
Creation Date: Sun Jan 30 2022 12:23 PM
@author: Sakibul Islam Sazzad

Codes are modified from Lucas Arenstein

refs: Ekert, Artur K. "Quantum cryptography based on Bell's theorem" - 1991
      https://github.com/kardashin/E91_protocol/blob/master/E91_tutorial/E91_tutorial.ipynb
"""
import sys
import math
import random

import re


from qiskit import QuantumCircuit, execute, Aer, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from quantuminspire.qiskit import QI

#email, password = "sakibulislamsazzad@gmail.com", ""
#QI.set_authentication_details(email=email, password=pasword)
#backend = QI.get_backend('QX single-node simulator')

In [2]:
#in E91 protocol Charles create a singlet state
#if Eve exist in the channel, she will intercepts entangled states creatd by Charles
#measures them according to the projection 
#and send them Alice and Bob
def initial_setup():
    qr = QuantumRegister(2)
    cr = ClassicalRegister(4)
    
    ## Alice's measurement circuits
    
    # measure the spin projection of Alice's qubit onto the a_1 direction (X basis)
    measureA1 = QuantumCircuit(2,4)
    measureA1.h(0)
    measureA1.measure(0,0)
    
    # measure the spin projection of Alice's qubit onto the a_2 direction (W basis)
    measureA2 = QuantumCircuit(2,4)
    measureA2.s(0)
    measureA2.h(0)
    measureA2.t(0)
    measureA2.h(0)
    measureA2.measure(0,0)
    
    # measure the spin projection of Alice's qubit onto the a_3 direction (standard Z basis)
    measureA3 = QuantumCircuit(2,4)
    measureA3.measure(0,0)
    
    ## Bob's measurement circuits
    
    # measure the spin projection of Bob's qubit onto the b_1 direction (W basis)
    measureB1 = QuantumCircuit(2,4)
    measureB1.s(1)
    measureB1.h(1)
    measureB1.t(1)
    measureB1.h(1)
    measureB1.measure(1,1)
    
    # measure the spin projection of Bob's qubit onto the b_2 direction (standard Z basis)
    measureB2 = QuantumCircuit(2,4)
    measureB2.measure(1,1)
    
    # measure the spin projection of Bob's qubit onto the b_3 direction (V basis)
    measureB3 = QuantumCircuit(2,4)
    measureB3.s(1)
    measureB3.h(1)
    measureB3.tdg(1)
    measureB3.h(1)
    measureB3.measure(1,1)
    
    
    #spin projection of Alice qubit in W basis
    measureEA2= QuantumCircuit(2,4)
    measureEA2.s(0)
    measureEA2.h(0)
    measureEA2.t(0)
    measureEA2.h(0)
    measureEA2.measure(0,2)


#spin projection of Alice qubit in Z basis
    measureEA3= QuantumCircuit(2,4)
    measureEA3.measure(0,2)

#spin projection of Bob qubit in W basis
    measureEB1= QuantumCircuit(2,4)
    measureEB1.s(1)
    measureEB1.h(1)
    measureEB1.t(1)
    measureEB1.h(1)
    measureEB1.measure(1,3)

#spin projection of Bob qubit in Z basis
    measureEB2= QuantumCircuit(2,4)
    measureEB2.measure(1,3)

    
    ## Lists of measurement circuits
    aliceMeasurements = [measureA1, measureA2, measureA3]
    bobMeasurements = [measureB1, measureB2, measureB3]
    eveMeasurements = [measureEA2, measureEA3, measureEB1, measureEB2]
    
    #Alice and Bob record the results of their measurements as bits of the strings a and a'
    abPatterns = [
        re.compile('..00$'), # search for the '..00' output (Alice obtained -1 and Bob obtained -1)
        re.compile('..01$'), # search for the '..01' output
        re.compile('..10$'), # search for the '..10' output (Alice obtained -1 and Bob obtained 1)
        re.compile('..11$')  # search for the '..11' output
    ]
    
    ePatterns = [
        re.compile('00..$'), # search for the '00..' result (Eve obtained the results -1 and -1 for Alice's and Bob's qubits)
        re.compile('01..$'), # search for the '01..' result (Eve obtained the results 1 and -1 for Alice's and Bob's qubits)
        re.compile('10..$'),
        re.compile('11..$')  
    ]

    
    return aliceMeasurements, bobMeasurements, eveMeasurements, abPatterns, ePatterns

In [3]:
#list of Eve measurement choices
numberOfSinglets = 500

In [4]:
#copied from Lucas 
def run_circuits(numberOfSinglets, aliceMeasurements, bobMeasurements):
    aliceMeasurementChoices = [random.randint(1, 3) for i in range(numberOfSinglets)] # string b of Alice
    bobMeasurementChoices = [random.randint(1, 3) for i in range(numberOfSinglets)] # string b' of Bob
    eveMeasurementChoices = []
    
    circuits = [] # the list in which the created circuits will be stored
    circuitsNames = [] 

    for j in range(numberOfSinglets):
        if random.uniform(0, 1) <= 0.5: # in 50% of cases perform the WW measurement
            eveMeasurementChoices.append([0, 2])
        else: # in 50% of cases perform the ZZ measurement
            eveMeasurementChoices.append([1, 3])
    # create the name of the j-th circuit depending on Alice's, Bob's and Eve's choices of measurement
        circuitName = str(j) + ':A' + str(aliceMeasurementChoices[j]) + '_B' + str(bobMeasurementChoices[j] + 2) + '_E' + str(eveMeasurementChoices[j][0]) + str(eveMeasurementChoices[j][1] - 1)
    
    # create the joint measurement circuit
    # add Alice's and Bob's measurement circuits to the singlet state curcuit
        gen_circuit = QuantumCircuit(2, 4)
    
        gen_circuit.x(0)
        gen_circuit.x(1)
        gen_circuit.h(0)
        gen_circuit.cx(0,1)
        
        gen_circuit.compose(aliceMeasurements[aliceMeasurementChoices[j]-1], inplace=True)
        
        gen_circuit.compose(bobMeasurements[bobMeasurementChoices[j]-1], inplace=True)
        
        circuitsNames.append(circuitName)
        circuits.append(gen_circuit)

        
    backend = Aer.get_backend('statevector_simulator')
    results = execute(circuits,backend, shots=1).result().get_counts()
    
    return aliceMeasurementChoices, bobMeasurementChoices, results, eveMeasurementChoices

In [5]:
#Eve patterns 
ePatterns = [
    re.compile('00..$'), # search for the '00..' result (Eve obtained the results -1 and -1 for Alice's and Bob's qubits)
    re.compile('01..$'), # search for the '01..' result (Eve obtained the results 1 and -1 for Alice's and Bob's qubits)
    re.compile('10..$'),
    re.compile('11..$')  
]

#Alice and Bob patterns
abPatterns = [
        re.compile('..00$'), # search for the '..00' output (Alice obtained -1 and Bob obtained -1)
        re.compile('..01$'), # search for the '..01' output
        re.compile('..10$'), # search for the '..10' output (Alice obtained -1 and Bob obtained 1)
        re.compile('..11$')  # search for the '..11' output
    ]

In [9]:
def build_key(results, aliceMeasurementChoices, bobMeasurementChoices, abPatterns, ePatterns):
    aliceResults = [] # Alice's results (string a)
    bobResults = [] # Bob's results (string a')

# list of Eve's measurement results
# the elements in the 1-st column are the results obtaned from the measurements of Alice's qubits
# the elements in the 2-nd column are the results obtaned from the measurements of Bob's qubits
    eveResults = [] 

# recording the measurement results
    for j in range(numberOfSinglets):
    
        res = list(results[j])[0] # extract a key from the dict and transform it to str
    
    # Alice and Bob
        if abPatterns[0].search(res): # check if the key is '..00' (if the measurement results are -1,-1)
            aliceResults.append(-1) # Alice got the result -1 
            bobResults.append(-1) # Bob got the result -1
        if abPatterns[1].search(res):
            aliceResults.append(1)
            bobResults.append(-1)
        if abPatterns[2].search(res): # check if the key is '..10' (if the measurement results are -1,1)
            aliceResults.append(-1) # Alice got the result -1 
            bobResults.append(1) # Bob got the result 1
        if abPatterns[3].search(res): 
            aliceResults.append(1)
            bobResults.append(1)

    # Eve
        if ePatterns[0].search(res): # check if the key is '00..'
            eveResults.append([-1, -1]) # results of the measurement of Alice's and Bob's qubits are -1,-1
        if ePatterns[1].search(res):
            eveResults.append([1, -1])
        if ePatterns[2].search(res):
            eveResults.append([-1, 1])
        if ePatterns[3].search(res):
            eveResults.append([1, 1])
    
    
    aliceKey = [] # Alice's key string a
    bobKey = [] # Bob's key string a'
    eveKeys = [] # Eve's keys; the 1-st column is the key of Alice, and the 2-nd is the key of Bob

# comparing the strings with measurement choices (b and b')
    for j in range(numberOfSinglets):
    # if Alice and Bob measured the spin projections onto the a_2/b_1 or a_3/b_2 directions
        if (aliceMeasurementChoices[j] == 2 and bobMeasurementChoices[j] == 1) or (aliceMeasurementChoices[j] == 3 and bobMeasurementChoices[j] == 2):  
            aliceKey.append(aliceResults[j]) # record the i-th result obtained by Alice as the bit of the secret key k
            bobKey.append(-bobResults[j]) # record the multiplied by -1 i-th result obtained Bob as the bit of the secret key k'
            eveKeys.append([eveResults[j][0], -eveResults[j][1]]) # record the i-th bits of the keys of Eve 

    keyLength = len(aliceKey) # length of the secret skey
    
    abKeyMismatches = 0 # number of mismatching bits in the keys of Alice and Bob
    eaKeyMismatches = 0 # number of mismatching bits in the keys of Eve and Alice
    ebKeyMismatches = 0 # number of mismatching bits in the keys of Eve and Bob

    for j in range(keyLength):
        if aliceKey[j] != bobKey[j]: 
            abKeyMismatches += 1
        if eveKeys[j][0] != aliceKey[j]:
            eaKeyMismatches += 1
        if eveKeys[j][1] != bobKey[j]:
            ebKeyMismatches += 1
    eaKnowledge = (keyLength - eaKeyMismatches)/keyLength # Eve's knowledge of Bob's key
    ebKnowledge = (keyLength - ebKeyMismatches)/keyLength # Eve's knowledge of Alice's key
    
    return keyLength, abKeyMismatches, eaKeyMismatches, ebKeyMismatches, eaKnowledge, ebKnowledge

In [7]:
# function that calculates CHSH correlation value
def chsh_corr(results, aliceMeasurementChoices, bobMeasurementChoices, abPatterns):
    
    # lists with the counts of measurement results
    # each element represents the number of (-1,-1), (-1,1), (1,-1) and (1,1) results respectively
    countA1B1 = [0, 0, 0, 0] # XW observable
    countA1B3 = [0, 0, 0, 0] # XV observable
    countA3B1 = [0, 0, 0, 0] # ZW observable
    countA3B3 = [0, 0, 0, 0] # ZV observable

    for i in range(len(results)):

        res = list(results[i])[0]


        # if the spins of the qubits of the i-th singlet were projected onto the a_1/b_1 directions
        if (aliceMeasurementChoices[i] == 1 and bobMeasurementChoices[i] == 1):
            for j in range(4):
                if abPatterns[j].search(res):
                    countA1B1[j] += 1

        if (aliceMeasurementChoices[i] == 1 and bobMeasurementChoices[i] == 3):
            for j in range(4):
                if abPatterns[j].search(res):
                    countA1B3[j] += 1

        if (aliceMeasurementChoices[i] == 3 and bobMeasurementChoices[i] == 1):
            for j in range(4):
                if abPatterns[j].search(res):
                    countA3B1[j] += 1
                    
        # if the spins of the qubits of the i-th singlet were projected onto the a_3/b_3 directions
        if (aliceMeasurementChoices[i] == 3 and bobMeasurementChoices[i] == 3):
            for j in range(4):
                if abPatterns[j].search(res):
                    countA3B3[j] += 1
                    
    # number of the results obtained from the measurements in a particular basis
    total11 = sum(countA1B1)
    total13 = sum(countA1B3)
    total31 = sum(countA3B1)
    total33 = sum(countA3B3)      
                    
    # expectation values of XW, XV, ZW and ZV observables (2)
    expect11 = (countA1B1[0] - countA1B1[1] - countA1B1[2] + countA1B1[3])/total11 # -1/sqrt(2)
    expect13 = (countA1B3[0] - countA1B3[1] - countA1B3[2] + countA1B3[3])/total13 # 1/sqrt(2)
    expect31 = (countA3B1[0] - countA3B1[1] - countA3B1[2] + countA3B1[3])/total31 # -1/sqrt(2)
    expect33 = (countA3B3[0] - countA3B3[1] - countA3B3[2] + countA3B3[3])/total33 # -1/sqrt(2) 
    
    corr = expect11 - expect13 + expect31 + expect33 # calculate the CHSC correlation value (3)
    
    return corr

In [10]:
#def main(n):
    
    #if n<10:
     #   return print("Too few digits to distribute, try at least a 10")
    
    #Obtain this result after a series of simulations
n= 500
lower_bound_singlets = 8*n
    
aliceMeasurements, bobMeasurements, eveMeasurements, abPatterns, ePatterns = initial_setup()
    
aliceMeasurementChoices, bobMeasurementChoices, results, eveMeasurementChoices = run_circuits(lower_bound_singlets, aliceMeasurements, bobMeasurements)
    
keyLength, abKeyMismatches, eaKeyMismatches, ebKeyMismatches, eaKnowledge, ebKnowledge = build_key(results, aliceMeasurementChoices, bobMeasurementChoices, abPatterns, ePatterns)
    
corr = chsh_corr(results, aliceMeasurementChoices, bobMeasurementChoices, abPatterns)
    
    #Print Results:
    # CHSH inequality test
print('CHSH correlation value: ' + str(round(corr, 3)) + '\n')

# Keys
print('Length of the key: ' + str(keyLength))
print('Number of mismatching bits: ' + str(abKeyMismatches) + '\n')

print('Eve\'s knowledge of Alice\'s key: ' + str(round(eaKnowledge * 100, 2)) + ' %')
print('Eve\'s knowledge of Bob\'s key: ' + str(round(ebKnowledge * 100, 2)) + ' %')
    #return

CHSH correlation value: -2.853

Length of the key: 115
Number of mismatching bits: 0

Eve's knowledge of Alice's key: 49.57 %
Eve's knowledge of Bob's key: 50.43 %


In [ ]:
if __name__ == "__main__":
    n = int(sys.argv[1])
    main(n)